<a href="https://colab.research.google.com/github/urieliram/tc_uc/blob/main/ETL_WEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lectura de archivos **CSV** de mercado eléctrico y conversión a formato **JSON** de Knueven2021



In [1]:
# Importar librerías necesarias
import io
import requests
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
import pandas as pd
import numpy as np
import re

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# ID del archivo en Google Drive
# https://drive.google.com/file/d/1dTFMHQlXXB4fw66WZTG9mj7N9xPDYG7p/view?usp=share_link
file_id = "1dTFMHQlXXB4fw66WZTG9mj7N9xPDYG7p"

# URL de descarga del archivo
url = f"https://drive.google.com/uc?export=download&id={file_id}"
url = f"https://drive.google.com/uc?id={file_id}"
import urllib.request

# Descargar el archivo desde la URL
urllib.request.urlretrieve(url, "archivo.txt")

# Abrir el archivo descargado en modo lectura
archivo = open("archivo.txt", "r")


# Leer el archivo línea por línea
# for linea in archivo:
#     print(linea)
print(type(archivo))

<class '_io.TextIOWrapper'>


In [2]:
# Abrir el archivo y leer todas las líneas
with open("archivo.txt") as archivo:
    lineas = archivo.readlines()

# Buscar los índices de las líneas que contienen los textos de interés
bus_data_inicio = lineas.index("%% bus data\n") + 1
bus_data_fin = lineas.index("];\n", bus_data_inicio)
generator_data_inicio = lineas.index("%% generator data\n", bus_data_fin) + 1
generator_data_fin = lineas.index("];\n", generator_data_inicio)
branch_data_inicio = lineas.index("%% branch data\n", generator_data_fin) + 1
branch_data_fin = lineas.index("];\n", branch_data_inicio)
generator_cost_data_inicio = lineas.index("%% generator cost data\n", branch_data_fin) + 1
generator_cost_data_fin = lineas.index("];\n", generator_cost_data_inicio)

# Separar el archivo en cuatro secciones según los índices obtenidos
bus_data = "".join(lineas[bus_data_inicio:bus_data_fin+1])
generator_data = "".join(lineas[generator_data_inicio:generator_data_fin+1])
branch_data = "".join(lineas[branch_data_inicio:branch_data_fin+1])
generator_cost_data = "".join(lineas[generator_cost_data_inicio:generator_cost_data_fin+1])

In [3]:
# Imprimir las secciones de datos obtenidas
print(bus_data.count('\n') + 1)
print(generator_data.count('\n') + 1)
print(branch_data.count('\n') + 1)
print(generator_cost_data.count('\n') + 1)

9245
1449
16053
1450


In [4]:
# Dividir la cadena en una lista de líneas
lineas = bus_data.split("\n")

# Eliminar la segunda línea de la lista de líneas
del lineas[1]  # Opcionalmente, puedes usar lineas.pop(1) en su lugar
lineas.pop(); lineas.pop() # Eliminar la última línea de la lista de líneas
bus_data = "\n".join(lineas) # Unir las líneas restantes en una sola cadena de texto

# Dividir la cadena en una lista de líneas
# lineas = cadena_texto.rsplit("\n", 1)
bus_data = bus_data.replace("%", "")
bus_data = bus_data.replace(";", "")	
bus_data = bus_data.replace("\t", ",")	

lineas = bus_data.split('\n')
for i in range(len(lineas)):
    lineas[i] = lineas[i][1:]
bus_data = '\n'.join(lineas)

#print(bus_data)
bus_data = StringIO(bus_data)

df = pd.read_csv(bus_data, sep=",")
print(df)
df.to_csv("bus_data.csv")

      bus_i  type      Pd    Qd   Gs     Bs  area        Vm         Va  \
0         1     1  378.40  98.5  0.0  16.61     0  1.007206 -36.856998   
1         2     2    0.00   0.0  0.0   0.00     0  1.031734 -12.028001   
2         3     1  151.00  48.8  0.0   4.69     0  1.017843 -21.794234   
3         4     1  171.41  23.4  0.0   2.10     0  1.020147  -7.224254   
4         5     1   45.95  14.0  0.0  13.34     0  0.989775 -49.871783   
...     ...   ...     ...   ...  ...    ...   ...       ...        ...   
9236   9237     1   -0.00  -0.0  0.0   1.27     0  1.047699 -23.228502   
9237   9238     2    0.00   0.0  0.0   0.00     0  1.097633  15.484001   
9238   9239     2    0.00   0.0  0.0   0.00     0  1.017601   6.125470   
9239   9240     1    0.00  -0.0  0.0   3.66     0  0.960374 -31.768249   
9240   9241     1    0.00  -0.0  0.0   0.36     0  1.044245  -9.735227   

      baseKV  zone  Vmax  Vmin  
0        220     6   1.1   0.9  
1        154     1   1.1   0.9  
2        220